In [1]:
# Skriptas sukuria zive duomenų analogą iš MIT duomenų, į sub-aplanką rec_dir įrašomi
# transformuoti EKG įrašai *.npa ir anotacijų failai *.json.
# # Išeities duomenys MIT_BIH atsisiunčiami iš aplanko mit-bih-arrhythmia-database-1.0.0
# Anotacijų failuose paliktos iš MIT tik anotacijos N,L,R,e,j,V,E,S,A,a,J,F,
# visas kitas perpavadiname į 'Q' 
# Pagrindinis sukūriamas aplankas - db_path = pvz. 'mit2zive-arrhythmia-database-1.0.0'.

# Programos planas:
# Nuskaitome MIT-BIH ekg įrašą
# Anotacijų mapingas į N,V,S,F,Q
# Pakeičiame diskretizavimo dažnį iš 360 į 200 Hz
# Perskaičiuojame anotacijos vietas
# Įrašome ekg ir anotacijas zive formatu į diską

# //////////////////////////////////////////////////////////////////////////////////////////////////////
#  
import pandas as pd
import numpy as np
import wfdb

import matplotlib.pyplot as plt
import json
import sys
from icecream import ic
from pathlib import Path
from neurokit2 import signal_resample

from mit2zive_util import create_dir
from mit2zive_util import ecg_record_plot
from mit2zive_util import plot_sign


# Maksimumo vietos lyginimui 
def max_place(subject, atr_sample, sign, sample_from, sample_to):  
    for i in range(len(atr_sample)):
        index_s = atr_sample[i]
        if (index_s < sample_from or index_s >sample_to):
            continue
        print(f"\n{i}, index_s: {index_s}  max: {sign[index_s]:.3f}")
        for j in range(-5,5):
            index_y = index_s+j
            formatted_float = "{:.3f}".format(sign[index_y])
            print(index_y, formatted_float, end =' ')
        

In [2]:
print("\nSukuriamas Zive duomenų analogas iš MIT duomenų")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'


# Aplankas su MIT-BIH duomenų rinkiniu
db_folder_mit = 'mit-bih-arrhythmia-database-1.0.0'

# Užduodamas pacientų įrašų sąrašas
records = np.array([124,209,228])
# records = np.array([100, 101, 103, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
# 200, 201, 202, 203, 205, 208, 209, 210, 212, 213, 214, 215, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234])

# MIT_BIH duomenų diskretizavimo dažnumas
fs_mit = 360

# ////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT-BIH duomenų rinkinį
db_path_mit = Path(Duomenu_aplankas, db_folder_mit)

print("Bendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("MIT duomenų aplankas: ", db_folder_mit)
print("Pacientų įrašų sąrašas:\n",records)
print("Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers")
print("207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF")
print("\nDiskretizavimo dažnis fs_mit: ", fs_mit)


# //////////////// Nurodomi MIT2ZIVE parametrai ////////////////////////////////

#  Aplankas su MIT2ZIVE duomenų rinkiniu
db_folder_mit2zive = 'ZIVE&MIT_TST'

# MIT2ZIVE duomenų diskretizavimo dažnumas
fs_zive = 200 # diskretizavimo dažnumas

# Paliekamų anotacijų sąrašas
annot_list = ['N','L','R','e','j','A','a','J','S','V','E','F','Q']

# ////////////////////////////////////////////////////////////////////////


# Nuoroda į MIT2ZIVE duomenų rinkinį
db_path_mit2zive = Path(Duomenu_aplankas, db_folder_mit2zive)

# Nuoroda į aplanką MIT2ZIVE EKG įrašams (.npa) ir anotacijoms (.json)
rec_dir = Path(db_path_mit2zive, 'records')

sant = fs_zive/fs_mit

print("\nAplankas transformuotiems duomenims: ", db_folder_mit2zive)

# Sukūriame aplanką transformuotiems iš MIT į Zive formatą duomenims
create_dir(rec_dir)

print("Diskretizavimo dažnis fs_zive: ", fs_zive)
print(f"fs_zive/fs_mit = {sant:.2f}")

print("Paliekamos įrašuose anotacijos:", annot_list)

# Vaizdavimo intervalas (sekundėmis)
Interv = 1

# Vaizduojama reikšmių
Reiksm_MIT_360 = Interv*fs_mit 
Reiksm_Zive_200 = Interv*fs_zive

# Kiek reikšmių nupjauname įrašų pradžioje
Tranc_MIT = 20000
  
# Kiek reikšmių nupjauname resamplintame įraše
Tranc_Resampl = int(Tranc_MIT*sant)

print(f'\nTranc_MIT, Tranc_Resampl = {Tranc_MIT, Tranc_Resampl} ')


Sukuriamas Zive duomenų analogas iš MIT duomenų
OS in my system :  win32
Bendras Zive duomenų aplankas:  C:\DI\Data\MIT&ZIVE
MIT duomenų aplankas:  mit-bih-arrhythmia-database-1.0.0
Pacientų įrašų sąrašas:
 [124 209 228]
Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers
207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF

Diskretizavimo dažnis fs_mit:  360

Aplankas transformuotiems duomenims:  ZIVE&MIT_TST
Directory 'C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\records' created successfully
Diskretizavimo dažnis fs_zive:  200
fs_zive/fs_mit = 0.56
Paliekamos įrašuose anotacijos: ['N', 'L', 'R', 'e', 'j', 'A', 'a', 'J', 'S', 'V', 'E', 'F', 'Q']

Tranc_MIT, Tranc_Resampl = (20000, 11111) 


In [3]:
# Ciklas per pacientų įrašus
for subject in records:

    # Nuskaitome įrašą
    subject_path = f'{db_path_mit}/{subject}'
    # ic(subject_path)

    # Įrašo skaitymo variantai
    # sequence, fields = wfdb.rdsamp(subject_path, channels=[0])
    # record = wfdb.rdrecord('sample-data/a103l')

# https://www.programmersought.com/article/28613723297/
    
     # Variantas 1 - skaitmeninės reikšmės
    # record = wfdb.rdrecord(subject_path, sampfrom=1,
    #                             channels=[0], physical=False)
    # sign_raw = record.d_signal[:,0]

    # Variantas 2 - fizinės reikšmės - perverstos į mV
    record = wfdb.rdrecord(subject_path, sampfrom=0,channels=[0], physical=True)
    sign_raw = record.p_signal[:,0]

    # https://wfdb.readthedocs.io/en/latest/wfdb.html

    len_sign = sign_raw.shape[0]
    print("\n")
    print(f"Pacientas iš MIT: {subject}  Reikšmių: {len_sign}")

    sign_transf = sign_raw
    # sign_transf = signal_filter(signal=sign_raw, sampling_rate=360,
    #  lowcut=0.1, method="butterworth", order=5)


# Nuskaitome originalaus įrašo anotacijas
    # https://wfdb.readthedocs.io/en/latest/wfdb.html 

    ann = wfdb.rdann(subject_path, 'atr', sampfrom=0, sampto=None, shift_samps=False)
    atr_sample = ann.sample
    atr_symbol = np.array(ann.symbol)

# Originalaus įrašo vizualizacija
# Vaizduodami nupjauname įrašo pradžios gabaliuką, užduodame vizualizacijos diapazoną
    sampfrom = Tranc_MIT
    sampto = Reiksm_MIT_360 + Tranc_MIT
    # plot_sign(sign_raw, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol,
    #  title= 'Pacientas ' + str(subject) +' Orig 360 Hz', figsize=(18,3))

# //////////////////////////////////// laikinas
# Maksimumo vietos lyginimui
    # print("\nsubject original:", subject)
    # max_place(subject, atr_sample, sign_raw, sampfrom, sampto)   
# //////////////////////////////////// laikinas

# Resampling iš 360 Hz (MIT) į 200 Hz (Zive)
    # if (subject == 100):
    #     print(sign_raw[:20])
    sign_transf = signal_resample(sign_raw, sampling_rate=fs_mit,
                              desired_sampling_rate=200, method="numpy")
    # if (subject == 100):
    #     print(sign_transf[:20])
    # https://neurokit2.readthedocs.io/en/latest/_modules/neurokit2/signal/signal_resample.html

    print(f"Pacientas Zive rinkiniui: {subject}  Reikšmių: {sign_transf.shape[0]}")

# Resampliname anotacijų vietas
    atr_sample = (atr_sample*sant).astype(int)

# Paliekame nepakeistas tik užduotas anotacijas iš annot_list, visas kitas perpavadiname į 'Q'
    for i in range(atr_sample.shape[0]):
        if atr_symbol[i] not in annot_list:
            atr_symbol[i] = 'Q'

# Resamplinto įrašo vizualizacija
    sampfrom = Tranc_Resampl
    sampto = Reiksm_Zive_200 + Tranc_Resampl
    # plot_sign(sign_transf, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol,
    #  title='Pacientas ' + str(subject) +' MIT Resampled 200 Hz', figsize=(18,3))

# //////////////////////////////////// laikinas
# Maksimumo vietos lyginimui
    # print("\nsubject resampled:", subject)
    # max_place(subject, atr_sample, sign_transf, sampfrom, sampto)   
# //////////////////////////////////// laikinas


# Anotacijos perrašomos json formatu:
    atr_list = [{"sampleIndex":int(atr_sample[i]),"annotationValue":atr_symbol[i]} for i in range(atr_sample.shape[0])]
    # print(atr_list[:20])

# Anotacijos įrašomos į diską
    file_path = Path(rec_dir, str(subject) + '.json')
    with open(file_path, 'w') as f:
        json.dump(atr_list, f)

# Įrašoma EKG
    file_path = Path(rec_dir, str(subject) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, sign_transf)

# SKAITYMO TESTAS

# Kontrolinis anotacijų skaitymas

    file_path = Path(rec_dir, str(subject) + '.json')
    df = pd.read_json(file_path, orient = 'records')
    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()
    # ic(atr_sample[:20], atr_symbol[:20])

    # print('sign_transf', sign_transf.shape)
    # print(sign_transf[:10])

# Kontrolinis EKG skaitymas
    file_path = Path(rec_dir, str(subject) + '.npy')
    with open(file_path, "rb") as f:
        sign_loaded = np.load(f) 
    # print(sign_loaded[:10])
    # print('sign_loaded', sign_loaded.shape)

# Originalaus įrašo vizualizacija naudojant wfdb analogą
    # plot_sign(sign_loaded, sampfrom, sampto, ann_samp=atr_sample, ann_symb=atr_symbol, 
    # title=str(subject) +' test', figsize=(18,3))

# Suformuojame info_create masyvą ir įrašome į diską

# Transformuojame pacientų sąrašą įrašymui į info_creat
subjects = records.tolist()
subjects = [str(x) for x in subjects]

info = {
    'db_folder':db_folder_mit2zive,
    'fs': fs_zive,
    'subj_list':subjects,
    'annot_list':annot_list
    }

file_name = Path(rec_dir,'info_create.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )

print("\nPabaiga.............")




Pacientas iš MIT: 124  Reikšmių: 650000
Pacientas Zive rinkiniui: 124  Reikšmių: 361111


Pacientas iš MIT: 209  Reikšmių: 650000
Pacientas Zive rinkiniui: 209  Reikšmių: 361111


Pacientas iš MIT: 228  Reikšmių: 650000
Pacientas Zive rinkiniui: 228  Reikšmių: 361111

Schemos parametrai įrašyti į failą:  C:\DI\Data\MIT&ZIVE\ZIVE&MIT_TST\records\info_create.json 


Pabaiga.............
